# Testing packed sequence

In [1]:
import pandas as pd
import os
import yaml
import matplotlib

%matplotlib inline

In [2]:
exp_dir = "../exps/test_packed/"

experiments = []

for path in os.scandir(exp_dir):
    with open(os.path.join(path, "config.yaml")) as f:
        exp_d = yaml.load(f)
    with open(os.path.join(path, "result.yaml")) as f:
        exp_d.update(yaml.load(f))
    with open(os.path.join(path, "train.word_accuracy")) as f:
        exp_d['train_acc'] = float(f.read())
    with open(os.path.join(path, "dev.word_accuracy")) as f:
        exp_d['dev_acc'] = float(f.read())
    with open(os.path.join(path, "test.word_accuracy")) as f:
        exp_d['test_acc'] = float(f.read())
    
    # strip paths
    for k, v in exp_d.items():
        if not isinstance(v, str):
            continue
        if "homes" in v:
            exp_d[k] = v.split("deep-morphology/")[1]
            
    experiments.append(exp_d)
    
experiments = pd.DataFrame(experiments)

In [3]:
params = ['packed', 'mask_pad_in_loss', 'pad_batch_level', 'pad_right']
result_cols = ['train_acc', 'dev_acc', 'test_acc']
experiments.groupby(params).agg(['max', 'min', 'mean', 'std'])[result_cols]

train_acc                    \
                                                        max     min      mean   
packed mask_pad_in_loss pad_batch_level pad_right                               
False  False            False           False        0.9830  0.7655  0.892147   
                                        True         0.9823  0.9032  0.945327   
                        True            False        0.9647  0.8028  0.925080   
                                        True         0.9721  0.9111  0.941093   
       True             False           False        0.9401  0.7315  0.866567   
                                        True         0.9699  0.8019  0.927873   
                        True            False        0.9560  0.8914  0.931633   
                                        True         0.9830  0.8819  0.936547   
True   False            False           True         0.9616  0.9248  0.944827   
                        True            True         0.9762  0.8511  0.926140   
       True             False           True         0.9650  0.8343  0.932793   
                        True            True         0.9683  0.8817  0.940413   

                                                            dev_acc         \
                                                        std     max    min   
packed mask_pad_in_loss pad_batch_level pad_right                            
False  False            False           False      0.067490   0.870  0.003   
                                        True       0.021219   0.886  0.798   
                        True            False      0.045637   0.875  0.769   
                                        True       0.020155   0.882  0.798   
       True             False           False      0.065933   0.860  0.024   
                                        True       0.040811   0.866  0.624   
                        True            False      0.018237   0.866  0.835   
                                        True       0.025962   0.869  0.788   
True   False            False           True       0.011296   0.879  0.817   
                        True            True       0.037029   0.884  0.804   
       True             False           True       0.032912   0.879  0.794   
                        True            True       0.019476   0.877  0.822   

                                                                       \
                                                       mean       std   
packed mask_pad_in_loss pad_batch_level pad_right                       
False  False            False           False      0.706667  0.230829   
                                        True       0.852267  0.026364   
                        True            False      0.838133  0.025102   
                                        True       0.854200  0.021588   
       True             False           False      0.684067  0.235137   
                                        True       0.826600  0.060819   
                        True            False      0.853667  0.008764   
                                        True       0.838267  0.024235   
True   False            False           True       0.856400  0.016304   
                        True            True       0.846600  0.020385   
       True             False           True       0.853533  0.019522   
                        True            True       0.856067  0.017036   

                                                  test_acc                   \
                                                       max    min      mean   
packed mask_pad_in_loss pad_batch_level pad_right                             
False  False            False           False        0.871  0.004  0.707600   
                                        True         0.882  0.776  0.854867   
                        True            False        0.873  0.785  0.844467   
                                        True         0.885  0.792  0.859467   
       True   

# Right padding vs. left padding

In [4]:
experiments[experiments.packed==False].groupby('pad_right').agg(['mean', 'max', 'min'])[result_cols]

train_acc                  dev_acc                test_acc         \
               mean    max     min      mean    max    min      mean    max   
pad_right                                                                     
False      0.903857  0.983  0.7315  0.770633  0.875  0.003  0.772983  0.873   
True       0.937710  0.983  0.8019  0.842833  0.886  0.624  0.849533  0.885   

                  
             min  
pad_right         
False      0.004  
True       0.621

# Packed vs. unpacked

In [5]:
experiments[experiments.pad_right==True].groupby('packed').agg(['mean', 'max', 'min'])[result_cols]

train_acc                   dev_acc                test_acc         \
            mean     max     min      mean    max    min      mean    max   
packed                                                                      
False   0.937710  0.9830  0.8019  0.842833  0.886  0.624  0.849533  0.885   
True    0.936043  0.9762  0.8343  0.853150  0.884  0.794  0.860983  0.881   

               
          min  
packed         
False   0.621  
True    0.809

# Pad to the longest sequence in the batch (batch level) vs. longest sequence in dataset

In [6]:
experiments[experiments.pad_right==True].groupby('pad_batch_level').agg(['mean', 'max', 'min'])[result_cols]

train_acc                   dev_acc                test_acc  \
                     mean     max     min      mean    max    min      mean   
pad_batch_level                                                               
False            0.937705  0.9823  0.8019  0.847200  0.886  0.624  0.852750   
True             0.936048  0.9830  0.8511  0.848783  0.884  0.788  0.857767   

                               
                   max    min  
pad_batch_level                
False            0.882  0.621  
True             0.885  0.792

In [7]:
experiments[experiments.pad_right==True].groupby(['packed', 'pad_batch_level']).agg(['mean', 'max', 'min'])[result_cols]

train_acc                   dev_acc                \
                            mean     max     min      mean    max    min   
packed pad_batch_level                                                     
False  False            0.936600  0.9823  0.8019  0.839433  0.886  0.624   
       True             0.938820  0.9830  0.8819  0.846233  0.882  0.788   
True   False            0.938810  0.9650  0.8343  0.854967  0.879  0.794   
       True             0.933277  0.9762  0.8511  0.851333  0.884  0.804   

                        test_acc                
                            mean    max    min  
packed pad_batch_level                          
False  False            0.844567  0.882  0.621  
       True             0.854500  0.885  0.792  
True   False            0.860933  0.879  0.809  
       True             0.861033  0.881  0.821

# Running time

In [8]:
experiments[experiments.pad_right==True].groupby(['packed', 'pad_batch_level']).agg(['mean', 'max', 'min'])['running_time']

mean         max         min
packed pad_batch_level                                    
False  False            392.110964  522.181896  208.352054
       True             304.455103  408.230499  211.952417
True   False            397.105736  584.338191  133.232946
       True             300.008933  400.842926  174.181087

## Per epoch running time

In [9]:
experiments['epochs_run'] = experiments['train_loss'].apply(len)
experiments['time_per_epoch'] = experiments['running_time'] / experiments['epochs_run']
experiments[experiments.pad_right==True].groupby(['packed', 'pad_batch_level']).agg(['mean', 'max', 'min'])['time_per_epoch']

mean        max       min
packed pad_batch_level                                
False  False            11.176217  15.153594  8.882848
       True              8.957555  11.847963  6.906339
True   False            11.229029  15.792924  9.392391
       True              8.955741   9.908076  7.171449